# Computing derived indicators from timeseries data

This notebook was presented by Daniel Huppmann as a "code-and-tell" session in the MESSAGEix weekly meeting on June 16, 2021.

The tutorial is based on the advanced assignment of the Modelling Lab of
the **Climate Risks Academy 2021** organized by the European University Institute (EUI)
Florence School of Banking and Finance in cooperation with Oliver Wyman.

The scenario data used here is taken from the [NGFS Scenario Explorer hosted by IIASA](https://data.ece.iiasa.ac.at/ngfs), Phase 2 (June 2021).

See [here](https://github.com/danielhuppmann/climate-risks-academy-2021)
for the full assignment (and template solutions) from the Modelling Lab class.

### Requirements

You can install pyam using the following command -
note the subtle naming difference on [pypi.org](https://pypi.org/project/pyam-iamc/).

```
pip install pyam-iamc
```


In [ ]:
import pyam

In [ ]:
df = pyam.IamDataFrame("ngfs_snapshot.csv")

## Show a summary of the scenario data

Just calling the **IamDataFrame** prints an overview of all dimensions and coordinates.

In [ ]:
df

Because there are more scenarios and variables than can be displayed in one line, the summary only shows a few items.

We can easily display all items of a dimension or coordinate individually.

In [ ]:
df.scenario

In [ ]:
df.variable

## Compute the amount of primary energy that is not coal

### Step 1: Compute total primary energy

For the first section of this example, we want to compute the amount of primary energy that is not supplied from coal for each scenario. For that, we first have to compute the aggregate primary energy supply.

By default, the [aggregate()](https://pyam-iamc.readthedocs.io/en/stable/api/iamdataframe.html#pyam.IamDataFrame.aggregate) method
takes all components of the given variable, in this case `Primary Energy|*`.<br />
It returns a new **IamDataFrame** - and the cell displays the summary. You will see that the object has exactly one variable now.

In [ ]:
df.aggregate("Primary Energy")

Or you can use the [timeseries()](https://pyam-iamc.readthedocs.io/en/stable/api/iamdataframe.html#pyam.IamDataFrame.timeseries) method to show the timeseries data in wide format.

In [ ]:
df.aggregate("Primary Energy").timeseries()

It is often convenient to directly append computed timeseries data to the original object. You can use the `append=True` keyword argument.

In [ ]:
df.aggregate("Primary Energy", append=True)

When displaying the variables of the **IamDataFrame**, there is now an additional variable `Primary Energy`.

In [ ]:
df.variable

### Step 2: Compute total primary energy 

Now, we can compute the amount of primary energy that is not coal.

All algebraic-operations functions follow the syntax:

```
df.<method>(a, b, c) => a <op> b = c
```

Note that in simple cases, **pyam** will try to keep the unit consistent during the operation.

First, we subtract coal from total primary energy and draw a simple [plot](https://pyam-iamc.readthedocs.io/en/stable/api/plotting.html).

In [ ]:
df.subtract("Primary Energy", "Primary Energy|Coal", "Non-Coal").plot()

Next, we can also compute the share of coal relative to total primary energy, and again draw the plot.

In [ ]:
df.divide("Primary Energy|Coal", "Primary Energy", "Non-Coal").plot()

Note that **pyam** has automatically changed the unit on the y-axis. Dividing `EJ/yr` by `EJ/yr` results in a dimensionless value.

As a third step, you may be annoyed by the legend being automatically placed in the figure, which makes it difficult to interpret the plot.<br />
You can simply pass a legend-location dictionary as keyword argument to the plot!

In [ ]:
(
    df.divide("Primary Energy|Coal", "Primary Energy", "Non-Coal")
    .plot(legend=dict(loc="outside bottom"))
)

## Compute ratio of energy sources between different scenarios

So far, we used the algebraic operations on the (default) *variable* axis.
But **pyam** also supports these operations on any other axis of the timeseries data!

Now, we compute the relative indicator between the *Net Zero 2050* and the *Current Policies*,
and again plot the resulting timeseries data.

In [ ]:
(
    df.divide("Net Zero 2050", "Current Policies", "diff", axis="scenario")
    .plot(legend=dict(loc="outside right"))
)

As a final illustration, this tutorials shows how to use **matplotlib** and **pyam** to create several plots next to each other.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
scenario = [
    "Below 2°C",
    "Delayed transition",
    "Divergent Net Zero",
    "Nationally Determined Contributions (NDCs) ",
    "Net Zero 2050",
]

baseline = "Current Policies "

# We first create a matplotlib figure with several "axes" objects (i.e., individual plots)
fig, ax = plt.subplots(1, len(scenario), figsize=(15, 5), sharey=True)

# We iterate over all scenarios
for i, s in enumerate(scenario):
    (
        df.divide(s, "Current Policies", "diff", axis="scenario")
        # We instruct pyam to "draw" the plot directly in a specific plot (axes) of the matplotlib figure
        .plot(ax=ax[i], legend=dict(loc="outside right") if i == len(scenario) - 1 else False)
    )
    
    # We specifically set the scenario name as plot title instead of the pyam default
    ax[i].set_title(s)

<div class="alert alert-info">
    
**Curious about more pyam features?** Check out the all the pyam tutorials on our [ReadTheDocs page](https://pyam-iamc.readthedocs.io/en/stable/tutorials.html)!

</div>